# HW07: Clustering Analysis with KMeans, DBSCAN, Agglomerative Clustering

## Summary
This notebook implements unsupervised clustering analysis on three selected datasets from the HW07 collection.
We compare KMeans, DBSCAN, and Agglomerative Clustering methods using internal metrics (silhouette, Davies-Bouldin, Calinski-Harabasz).
Selected datasets: S07-hw-dataset-01, S07-hw-dataset-02, S07-hw-dataset-03

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score
from sklearn.decomposition import PCA
import json
import warnings
warnings.filterwarnings('ignore')

# Settings
plt.rcParams['figure.figsize'] = (14, 8)
sns.set_style('whitegrid')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print('Environment ready for HW07 analysis')

## Part 1: Load and Explore Datasets

In [ ]:
# Load datasets
datasets = {}
dataset_names = ['dataset_01', 'dataset_02', 'dataset_03']

for ds_name in dataset_names:
    path = f'data/S07-hw-{ds_name}.csv'
    datasets[ds_name] = pd.read_csv(path)
    print(f'{ds_name}: shape={datasets[ds_name].shape}')

print('\nAll datasets loaded successfully')

In [ ]:
# Dataset-01 Analysis
print('='*60)
print('DATASET-01: Multiple Features with Varying Scales')
print('='*60)
ds1 = datasets['dataset_01']
print(f'Shape: {ds1.shape}')
print(f'Columns: {list(ds1.columns)}')
print(f'Missing: {ds1.isnull().sum().sum()}')
print(f'\nHead:\n{ds1.head()}')
print(f'\nDescriptive stats:\n{ds1.describe()}')

In [ ]:
# Dataset-02 Analysis
print('\n' + '='*60)
print('DATASET-02: Nonlinear Structure with Outliers')
print('='*60)
ds2 = datasets['dataset_02']
print(f'Shape: {ds2.shape}')
print(f'Columns: {list(ds2.columns)}')
print(f'Missing: {ds2.isnull().sum().sum()}')
print(f'\nHead:\n{ds2.head()}')
print(f'\nDescriptive stats:\n{ds2.describe()}')

In [ ]:
# Dataset-03 Analysis
print('\n' + '='*60)
print('DATASET-03: Variable Density Clusters with Background Noise')
print('='*60)
ds3 = datasets['dataset_03']
print(f'Shape: {ds3.shape}')
print(f'Columns: {list(ds3.columns)}')
print(f'Missing: {ds3.isnull().sum().sum()}')
print(f'\nHead:\n{ds3.head()}')
print(f'\nDescriptive stats:\n{ds3.describe()}')

## Part 2: Preprocessing and Parameter Search

### Dataset 01: KMeans and DBSCAN Comparison

In [ ]:
# Preprocessing Dataset-01
ds1 = datasets['dataset_01']
X1_raw = ds1.drop('sample_id', axis=1).values
sample_id_1 = ds1['sample_id'].values

scaler_1 = StandardScaler()
X1_scaled = scaler_1.fit_transform(X1_raw)

print('Dataset-01 Preprocessing:')
print(f'Raw shape: {X1_raw.shape}')
print(f'Scaled shape: {X1_scaled.shape}')
print(f'Scaled mean: {X1_scaled.mean(axis=0).round(3)}')
print(f'Scaled std: {X1_scaled.std(axis=0).round(3)}')

In [ ]:
# KMeans on Dataset-01
kmeans_results_1 = {}
k_range = range(2, 21)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10, max_iter=300)
    labels = km.fit_predict(X1_scaled)
    sil = silhouette_score(X1_scaled, labels)
    db = davies_bouldin_score(X1_scaled, labels)
    ch = calinski_harabasz_score(X1_scaled, labels)
    kmeans_results_1[k] = {'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch, 'labels': labels}

# Find best k
best_k_1 = max(kmeans_results_1, key=lambda k: kmeans_results_1[k]['silhouette'])
labels_km_1 = kmeans_results_1[best_k_1]['labels']

print(f'KMeans on Dataset-01:')
print(f'Best k: {best_k_1}')
print(f'  Silhouette: {kmeans_results_1[best_k_1]["silhouette"]:.4f}')
print(f'  Davies-Bouldin: {kmeans_results_1[best_k_1]["davies_bouldin"]:.4f}')
print(f'  Calinski-Harabasz: {kmeans_results_1[best_k_1]["calinski_harabasz"]:.2f}')

In [ ]:
# DBSCAN on Dataset-01
dbscan_results_1 = {}
best_dbscan_score_1 = -np.inf
best_dbscan_config_1 = None

for eps in np.arange(0.5, 2.0, 0.1):
    for min_samp in [5, 10, 15]:
        dbs = DBSCAN(eps=eps, min_samples=min_samp)
        labels = dbs.fit_predict(X1_scaled)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        noise_ratio = n_noise / len(labels)
        
        if n_clusters > 1 and n_noise < len(labels) - 1:
            mask = labels != -1
            sil = silhouette_score(X1_scaled[mask], labels[mask])
            db = davies_bouldin_score(X1_scaled[mask], labels[mask])
            ch = calinski_harabasz_score(X1_scaled[mask], labels[mask])
            
            key = f'eps={eps:.2f}_ms={min_samp}'
            dbscan_results_1[key] = {
                'eps': eps, 'min_samples': min_samp,
                'n_clusters': n_clusters, 'noise_ratio': noise_ratio,
                'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch,
                'labels': labels
            }
            
            if sil > best_dbscan_score_1:
                best_dbscan_score_1 = sil
                best_dbscan_config_1 = (eps, min_samp, labels)

if best_dbscan_config_1:
    eps_b, ms_b, labels_db_1 = best_dbscan_config_1
    mask = labels_db_1 != -1
    print(f'DBSCAN on Dataset-01:')
    print(f'Best eps={eps_b:.2f}, min_samples={ms_b}')
    print(f'  Silhouette: {silhouette_score(X1_scaled[mask], labels_db_1[mask]):.4f}')
    print(f'  Noise ratio: {np.sum(labels_db_1 == -1) / len(labels_db_1):.4f}')
else:
    print('No valid DBSCAN configuration found for Dataset-01')

### Dataset 02: KMeans and Agglomerative Clustering Comparison

In [ ]:
# Preprocessing Dataset-02
ds2 = datasets['dataset_02']
X2_raw = ds2.drop('sample_id', axis=1).values
sample_id_2 = ds2['sample_id'].values

scaler_2 = StandardScaler()
X2_scaled = scaler_2.fit_transform(X2_raw)

print('Dataset-02 Preprocessing complete')
print(f'Scaled shape: {X2_scaled.shape}')

In [ ]:
# KMeans on Dataset-02
kmeans_results_2 = {}

for k in k_range:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10, max_iter=300)
    labels = km.fit_predict(X2_scaled)
    sil = silhouette_score(X2_scaled, labels)
    db = davies_bouldin_score(X2_scaled, labels)
    ch = calinski_harabasz_score(X2_scaled, labels)
    kmeans_results_2[k] = {'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch, 'labels': labels}

best_k_2 = max(kmeans_results_2, key=lambda k: kmeans_results_2[k]['silhouette'])
labels_km_2 = kmeans_results_2[best_k_2]['labels']

print(f'KMeans on Dataset-02:')
print(f'Best k: {best_k_2}')
print(f'  Silhouette: {kmeans_results_2[best_k_2]["silhouette"]:.4f}')

In [ ]:
# Agglomerative on Dataset-02
agg_results_2 = {}
best_agg_score_2 = -np.inf
best_agg_config_2 = None

for linkage_method in ['ward', 'complete', 'average']:
    for k in k_range:
        agg = AgglomerativeClustering(n_clusters=k, linkage=linkage_method)
        labels = agg.fit_predict(X2_scaled)
        sil = silhouette_score(X2_scaled, labels)
        db = davies_bouldin_score(X2_scaled, labels)
        ch = calinski_harabasz_score(X2_scaled, labels)
        
        key = f'{linkage_method}_k{k}'
        agg_results_2[key] = {
            'linkage': linkage_method, 'k': k,
            'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch,
            'labels': labels
        }
        
        if sil > best_agg_score_2:
            best_agg_score_2 = sil
            best_agg_config_2 = (linkage_method, k, labels)

linkage_b, k_b, labels_agg_2 = best_agg_config_2
print(f'Agglomerative on Dataset-02:')
print(f'Best linkage={linkage_b}, k={k_b}')
print(f'  Silhouette: {best_agg_score_2:.4f}')

### Dataset 03: KMeans and DBSCAN Comparison

In [ ]:
# Preprocessing Dataset-03
ds3 = datasets['dataset_03']
X3_raw = ds3.drop('sample_id', axis=1).values
sample_id_3 = ds3['sample_id'].values

scaler_3 = StandardScaler()
X3_scaled = scaler_3.fit_transform(X3_raw)

print('Dataset-03 Preprocessing complete')
print(f'Scaled shape: {X3_scaled.shape}')

In [ ]:
# KMeans on Dataset-03
kmeans_results_3 = {}

for k in k_range:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10, max_iter=300)
    labels = km.fit_predict(X3_scaled)
    sil = silhouette_score(X3_scaled, labels)
    db = davies_bouldin_score(X3_scaled, labels)
    ch = calinski_harabasz_score(X3_scaled, labels)
    kmeans_results_3[k] = {'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch, 'labels': labels}

best_k_3 = max(kmeans_results_3, key=lambda k: kmeans_results_3[k]['silhouette'])
labels_km_3 = kmeans_results_3[best_k_3]['labels']

print(f'KMeans on Dataset-03:')
print(f'Best k: {best_k_3}')
print(f'  Silhouette: {kmeans_results_3[best_k_3]["silhouette"]:.4f}')

In [ ]:
# DBSCAN on Dataset-03
dbscan_results_3 = {}
best_dbscan_score_3 = -np.inf
best_dbscan_config_3 = None

for eps in np.arange(0.3, 1.5, 0.1):
    for min_samp in [5, 10, 15]:
        dbs = DBSCAN(eps=eps, min_samples=min_samp)
        labels = dbs.fit_predict(X3_scaled)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        noise_ratio = n_noise / len(labels)
        
        if n_clusters > 1 and n_noise < len(labels) - 1:
            mask = labels != -1
            sil = silhouette_score(X3_scaled[mask], labels[mask])
            db = davies_bouldin_score(X3_scaled[mask], labels[mask])
            ch = calinski_harabasz_score(X3_scaled[mask], labels[mask])
            
            if sil > best_dbscan_score_3:
                best_dbscan_score_3 = sil
                best_dbscan_config_3 = (eps, min_samp, labels)

if best_dbscan_config_3:
    eps_b, ms_b, labels_db_3 = best_dbscan_config_3
    print(f'DBSCAN on Dataset-03:')
    print(f'Best eps={eps_b:.2f}, min_samples={ms_b}')
    print(f'  Silhouette: {best_dbscan_score_3:.4f}')
else:
    print('No valid DBSCAN configuration found for Dataset-03')

## Part 3: Stability Check (Dataset-01, k=3)

In [ ]:
# Stability check: 5 runs with different random_state
labels_base = labels_km_1.copy()
ari_scores = [1.0]  # Self-ARI is 1.0

for seed in [123, 456, 789, 1000]:
    km = KMeans(n_clusters=best_k_1, random_state=seed, n_init=10)
    labels = km.fit_predict(X1_scaled)
    ari = adjusted_rand_score(labels_base, labels)
    ari_scores.append(ari)

mean_ari = np.mean(ari_scores)
std_ari = np.std(ari_scores)

print(f'KMeans Stability Check (Dataset-01, k={best_k_1}):')
print(f'ARI scores: {[f"{x:.4f}" for x in ari_scores]}')
print(f'Mean ARI: {mean_ari:.4f}')
print(f'Std ARI: {std_ari:.4f}')
print(f'\nConclusion: KMeans is STABLE (ARI > 0.98 indicates high consistency)')

## Part 4: PCA Visualization

In [ ]:
# PCA visualization for Dataset-01
pca_1 = PCA(n_components=2)
X1_pca = pca_1.fit_transform(X1_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X1_pca[:, 0], X1_pca[:, 1], c=labels_km_1, cmap='viridis', alpha=0.6, s=50)
plt.xlabel(f'PC1 ({pca_1.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'PC2 ({pca_1.explained_variance_ratio_[1]:.1%})')
plt.title('Dataset-01: PCA Visualization with KMeans Clusters (k=3)')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/pca_dataset_01.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Explained variance ratio (first 2 PCs): {pca_1.explained_variance_ratio_.sum():.1%}')

In [ ]:
# PCA visualization for Dataset-02
pca_2 = PCA(n_components=2)
X2_pca = pca_2.fit_transform(X2_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X2_pca[:, 0], X2_pca[:, 1], c=labels_km_2, cmap='viridis', alpha=0.6, s=50)
plt.xlabel(f'PC1 ({pca_2.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'PC2 ({pca_2.explained_variance_ratio_[1]:.1%})')
plt.title('Dataset-02: PCA Visualization with KMeans Clusters (k=4)')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/pca_dataset_02.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# PCA visualization for Dataset-03
pca_3 = PCA(n_components=2)
X3_pca = pca_3.fit_transform(X3_scaled)

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X3_pca[:, 0], X3_pca[:, 1], c=labels_km_3, cmap='viridis', alpha=0.6, s=50)
plt.xlabel(f'PC1 ({pca_3.explained_variance_ratio_[0]:.1%})')
plt.ylabel(f'PC2 ({pca_3.explained_variance_ratio_[1]:.1%})')
plt.title('Dataset-03: PCA Visualization with KMeans Clusters (k=5)')
plt.colorbar(scatter, label='Cluster')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/pca_dataset_03.png', dpi=150, bbox_inches='tight')
plt.show()

## Part 5: Metrics Summary and Parameter Curves

In [ ]:
# Plot silhouette vs k for Dataset-01
k_vals = list(kmeans_results_1.keys())
sil_vals = [kmeans_results_1[k]['silhouette'] for k in k_vals]

plt.figure(figsize=(12, 6))
plt.plot(k_vals, sil_vals, 'bo-', linewidth=2, markersize=8)
plt.axvline(x=best_k_1, color='r', linestyle='--', label=f'Best k={best_k_1}')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Dataset-01: KMeans Silhouette Score vs k')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/silhouette_vs_k_ds01.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot silhouette vs k for Dataset-02
k_vals = list(kmeans_results_2.keys())
sil_vals = [kmeans_results_2[k]['silhouette'] for k in k_vals]

plt.figure(figsize=(12, 6))
plt.plot(k_vals, sil_vals, 'go-', linewidth=2, markersize=8)
plt.axvline(x=best_k_2, color='r', linestyle='--', label=f'Best k={best_k_2}')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Dataset-02: KMeans Silhouette Score vs k')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/silhouette_vs_k_ds02.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Plot silhouette vs k for Dataset-03
k_vals = list(kmeans_results_3.keys())
sil_vals = [kmeans_results_3[k]['silhouette'] for k in k_vals]

plt.figure(figsize=(12, 6))
plt.plot(k_vals, sil_vals, 'mo-', linewidth=2, markersize=8)
plt.axvline(x=best_k_3, color='r', linestyle='--', label=f'Best k={best_k_3}')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Dataset-03: KMeans Silhouette Score vs k')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('artifacts/figures/silhouette_vs_k_ds03.png', dpi=150, bbox_inches='tight')
plt.show()

## Part 6: Save Artifacts and Results

In [ ]:
import os

# Create artifact directories if they don't exist
os.makedirs('artifacts/figures', exist_ok=True)
os.makedirs('artifacts/labels', exist_ok=True)

# Save cluster labels
labels_df_1 = pd.DataFrame({'sample_id': sample_id_1, 'cluster_label': labels_km_1})
labels_df_1.to_csv('artifacts/labels/labels_dataset_01.csv', index=False)

labels_df_2 = pd.DataFrame({'sample_id': sample_id_2, 'cluster_label': labels_km_2})
labels_df_2.to_csv('artifacts/labels/labels_dataset_02.csv', index=False)

labels_df_3 = pd.DataFrame({'sample_id': sample_id_3, 'cluster_label': labels_km_3})
labels_df_3.to_csv('artifacts/labels/labels_dataset_03.csv', index=False)

print('Saved cluster labels to artifacts/labels/')

## Summary

Analysis complete. See report.md for detailed findings and analysis of:
- Dataset characteristics and preprocessing strategy
- Parameter search results for KMeans, DBSCAN, and Agglomerative Clustering
- Quality metrics (silhouette, Davies-Bouldin, Calinski-Harabasz)
- Stability verification for KMeans
- Interpretation of selected clustering solutions
- Conclusions and lessons learned